In [2]:
!pip install pandas scikit-learn gradio


In [3]:
import pandas as pd

# If you have your own dataset, upload using this
from google.colab import files
uploaded = files.upload()

# Example filename: 'Crop_recommendation.csv'
data = pd.read_csv("Crop_recommendation.csv")
data.head()


Saving Crop_recommendation.csv to Crop_recommendation.csv


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split features and labels
X = data.drop("label", axis=1)
y = data["label"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"✅ Model trained successfully! Accuracy: {acc*100:.2f}%")


✅ Model trained successfully! Accuracy: 99.32%


In [5]:
import pickle

with open("crop_recommendation_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("🌾 Model saved as crop_recommendation_model.pkl")


🌾 Model saved as crop_recommendation_model.pkl


In [24]:
import gradio as gr
import pickle
import numpy as np
import pandas as pd

# --- Load model and dataset ---
model = pickle.load(open("crop_recommendation_model.pkl", "rb"))
data = pd.read_csv("Crop_recommendation.csv")

# --- Average fertilizer requirements ---
fertilizer_req = data.groupby("label")[["N", "P", "K"]].mean().round(1).to_dict("index")

# --- Fertilizer tips ---
fertilizer_tips = {
    "N": {
        "low": "💧 Add more Urea or Ammonium Sulphate (rich in Nitrogen)",
        "high": "⚠️ Reduce Nitrogen use — avoid overuse of Urea"
    },
    "P": {
        "low": "🌱 Use DAP or Bone Meal for Phosphorus",
        "high": "⚠️ Avoid excess DAP"
    },
    "K": {
        "low": "🌿 Add Muriate of Potash (MOP) or Wood Ash for Potassium",
        "high": "⚠️ Limit potash fertilizers"
    }
}

# --- Recommendation function ---
def recommend_with_fertilizer(N, P, K, temperature, humidity, ph, rainfall):
    if None in [N, P, K, temperature, humidity, ph, rainfall]:
        return "<p style='color:#2e4600;text-align:center;font-size:18px;'>⚠️ कृपया सभी मान भरें ताकि सटीक सिफारिश दी जा सके।<br>Please fill all input values to get recommendations.</p>"

    input_data = np.array([[N, P, K, temperature, humidity, ph, rainfall]])
    probs = model.predict_proba(input_data)[0]
    classes = model.classes_
    top3_idx = np.argsort(probs)[-3:][::-1]
    top3 = [classes[i].capitalize() for i in top3_idx]

    html = "<div style='display:flex;justify-content:center;flex-wrap:wrap;gap:20px;'>"
    for i, crop in enumerate(top3, start=1):
        rec = fertilizer_req[crop.lower()]
        html += f"""
        <div style='background-color:#ffffffc0;border-radius:15px;
                    padding:20px;text-align:center;
                    width:280px;box-shadow:0 4px 10px rgba(0,0,0,0.15);
                    color:#1b3a0a;'>
            <h2 style='color:#1b5e20;'>🏅 Rank {i}: {crop}</h2>
            <p><b>Ideal N:</b> {rec['N']}, <b>P:</b> {rec['P']}, <b>K:</b> {rec['K']}</p>
            <p><b>आदर्श N:</b> {rec['N']}, <b>P:</b> {rec['P']}, <b>K:</b> {rec['K']}</p>
        """

        for nutrient, user_val, ideal_val in zip(["N", "P", "K"], [N, P, K], [rec["N"], rec["P"], rec["K"]]):
            diff = user_val - ideal_val
            if diff < -5:
                html += f"<p style='color:#33691e;font-size:14px;'>🔻 {abs(diff):.1f} less {nutrient} — {fertilizer_tips[nutrient]['low']}</p>"
            elif diff > 5:
                html += f"<p style='color:#b71c1c;font-size:14px;'>🔺 {abs(diff):.1f} more {nutrient} — {fertilizer_tips[nutrient]['high']}</p>"
            else:
                html += f"<p style='color:#1b5e20;'>✅ {nutrient} balanced — पोषक तत्व संतुलित हैं</p>"

        html += "</div>"
    html += "</div>"
    return html


# --- Custom CSS ---
custom_css = """
.gradio-container {
    background: linear-gradient(rgba(255,255,255,0.6), rgba(255,255,255,0.6)),
                url('https://upload.wikimedia.org/wikipedia/commons/thumb/7/7f/A_group_of_Indian_women_farmers_Karnataka_Spice_Value_Chain_Development_2015.jpg/1280px-A_group_of_Indian_women_farmers_Karnataka_Spice_Value_Chain_Development_2015.jpg');
    background-size: cover;
    background-position: center;
    font-family: 'Poppins', sans-serif;
    color: #1b3a0a;
}
h1, h3, p, label {
    color: #1b3a0a !important;
}
input {
    background-color: #ffffff !important;
    border: 2px solid #2e7d32 !important;
    color: #1b3a0a !important;
    border-radius: 8px !important;
}
button {
    background-color: #2e7d32 !important;
    color: #ffffff !important;
    font-weight: bold;
    border-radius: 8px !important;
}
button:hover {
    background-color: #1b5e20 !important;
}
.footer {
    text-align: center;
    color: #1b5e20;
    font-size: 14px;
    margin-top: 15px;
}

/* Moving Top Bar */
.topbar {
    background-color: #1b5e20;
    color: #ffffff;
    padding: 12px 0;
    font-size: 18px;
    font-weight: 600;
    overflow: hidden;
    white-space: nowrap;
    position: fixed;
    top: 0;
    left: 0;
    right: 0;
    z-index: 999;
}
.topbar-content {
    display: inline-block;
    padding-left: 100%;
    animation: scroll 25s linear infinite;
}
@keyframes scroll {
    0% { transform: translateX(0); }
    100% { transform: translateX(-100%); }
}
"""

# --- UI Layout ---
with gr.Blocks(css=custom_css, title="SmartKheti") as app:
    # Moving Top Bar
    gr.HTML("""
    <div class="topbar">
        <div class="topbar-content">
            👨‍🌾 किसान सुझाव | Farmer’s Tips 🌾 — मिट्टी की नमी बनाए रखें 🌤 | उर्वरक का प्रयोग अनुशंसित मात्रा में करें 💧 | जल निकासी की व्यवस्था रखें 🚜 | जैविक खाद का उपयोग बढ़ाएं 🌱 | मिट्टी परीक्षण कराएं 🧪
        </div>
    </div>
    """)

    gr.HTML("<div style='height:70px;'></div>")

    # Centered title
    gr.HTML("<h1 style='text-align:center;color:#1b3a0a;'>🌾 SmartKheti 🌾</h1>")
    gr.Markdown("<p style='text-align:center;'>💡 Enter soil details below to get top crop and fertilizer advice</p>")

    with gr.Row():
        with gr.Column(scale=3):
            gr.Markdown("<h3 style='text-align:center;'>🧪 Soil Inputs / मिट्टी की जानकारी</h3>")
            with gr.Group():
                with gr.Row():
                    N = gr.Number(label="Nitrogen (N) / नाइट्रोजन", value=None)
                    P = gr.Number(label="Phosphorus (P) / फॉस्फोरस", value=None)
                with gr.Row():
                    K = gr.Number(label="Potassium (K) / पोटाश", value=None)
                    ph = gr.Number(label="Soil pH / मिट्टी का पीएच", value=None)
                with gr.Row():
                    temperature = gr.Number(label="Temperature (°C) / तापमान", value=None)
                    humidity = gr.Number(label="Humidity (%) / आर्द्रता", value=None)
                with gr.Row():
                    rainfall = gr.Number(label="Rainfall (mm) / वर्षा", value=None)

            predict_btn = gr.Button("🌱 Get Crop & Fertilizer Advice / सुझाव प्राप्त करें")
            output = gr.HTML("<p style='text-align:center;color:#1b3a0a;'>आपकी सिफारिशें यहाँ दिखाई देंगी।<br>Your recommendations will appear here.</p>")

            predict_btn.click(
                fn=recommend_with_fertilizer,
                inputs=[N, P, K, temperature, humidity, ph, rainfall],
                outputs=output
            )

    gr.Markdown("<div class='footer'>🌿 Developed to support Indian Farmers | भारतीय किसानों की सेवा में</div>")

app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://89bc3bff5bf07b550a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
